In [1]:
import pandas as pd
import missingno as msno
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
def toBool(any):
    return 1.00 if bool(any) else 0.00


# Read csv and convert test features to bool
df = pd.read_csv('./covid.csv', low_memory=False, converters={'tests_units': toBool})
# Display all columns
pd.set_option('display.max_columns', None)


## -- SPLIT FEATURES/LABELS --
labels = ['total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths','new_deaths', 'new_deaths_smoothed', 
                    'total_cases_per_million', 'new_cases_per_million','new_cases_smoothed_per_million', 
                    'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million']

# Remove international and world rows
df = df.drop(df[pd.isna(df.continent)].index)
# Remove rows where labels are NaN
for label in labels:
    df = df.drop(df[pd.isna(df[label])].index)
    

# Transform test features to 0 if there was not tested
df.loc[df['tests_units'] == 0.00, 
       ['new_tests', 'total_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'tests_per_case', 'positive_rate']] \
        = 0, 0, 0, 0, 0, 0, 0, 0


# Transform NaN values to the mean value of that feature
numeric_columns = df.select_dtypes(include='number').columns
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
df[numeric_columns] = pd.DataFrame(imputer.fit_transform(df[non_numeric_columns]))


# Remove rows where only Iso code etc. are filled
df.dropna(inplace=True)


# Create labels and features
non_numeric_features = df.drop(columns=labels, axis=1).select_dtypes(exclude='number').columns
features = df.drop(non_numeric_features, axis=1)

X = features
X = features['total_cases'].values.reshape(-1,1)
y = df['total_deaths']

# Split data between learning and test data
x_train,x_test,y_train,y_test = train_test_split(X, y,test_size=0.2, random_state=42)

print('train sizes: X, y')
print(x_train.shape, y_train.shape)
print('test sizes: X, y')
print(x_test.shape, y_test.shape)


# Train a model
ml = LinearRegression()
ml.fit(x_train, y_train)

# Check coefficients
print('Coefficients:')
print(ml.coef_)

# Check intercept
print('Intercept:')
print(ml.intercept_)


# Predict y
y_predicted = ml.predict(x_test)
print('Y predicted')
print(y_predicted)

# Validate predicted y
print('R2 Score:')
print(r2_score(y_test, y_predicted))
print('Mean squared error (MSE)', mean_squared_error(y_test, y_predicted))

# Display differences in numbers

pred_y_df = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_predicted, 'Difference': y_test - y_predicted})
print(pred_y_df[0:20])

# Visualize the validation results
plt.figure(figsize=(15, 10))
plt.scatter(y_test, y_predicted)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.title('Predicted deaths vs Actual deaths')
plt.show()



# Visualize results -> total cases vs total deaths
plt.figure(figsize=(15, 10))
plt.xlabel('Total cases')
plt.ylabel('Total deaths')
plt.title('PREDICTED: Total known cases vs total deaths')
plt.scatter(x_test, y_predicted, color='blue')
plt.show()

plt.figure(figsize=(15, 10))
plt.xlabel('Total cases')
plt.ylabel('Total deaths')
plt.title('ACTUAL: Total known cases vs total deaths')
plt.scatter(x_test, y_test, color='yellow')
plt.plot(x_test, y_predicted, color='red') # Regression line
plt.show()



ModuleNotFoundError: No module named 'pandas'